In [4]:
# test_imports in this cell, restart kernel whenever edits to modules are made

import cooltools
import cooler
import sys
import os
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import pyplot as plt
from matplotlib import gridspec
import numpy as np
import seaborn as sns



# Append the src directory to the path
current_dir = os.getcwd()
src_dir = os.path.abspath(os.path.join(current_dir, '..', 'src'))
if src_dir not in sys.path:
    sys.path.append(src_dir)

#import config based on which machine the jupyter notebook is running on
from configs.config_local import *
from preprocessing import *
from module_detection import *
from utils import *

def test_imports():
    print("testing config class")
    config = Config()
    print(config)

    print("testing 2d contacts preprocessing classes")
    query = HiCQuery(config, 'chr1', 1000000, '1Mb') #needs config, chrom and res/res0 for instantiation
    print(query)

    
    #other prints for imported classes

test_imports()

testing config class
testing 2d contacts preprocessing classes
HiC file loaded


In [ ]:
## check how much a/b from cooltools bigwig aligns with a/b from my code visually ##
#initialising parameters for single chromosome analysis

config = Config()
chrom = config.genomic_params.chromosomes[0]
current_res = config.genomic_params.resolutions[0]  # 1Mb for OE part
current_res_str = config.genomic_params.res_strs[0]  # 1Mb for OE part
query = HiCQuery(config, chrom, current_res, current_res_str)